# Видобуток та збагачення структурованих даних

### ТЕОРЕТИЧНА ЧАСТИНА ТА ПРИКЛАДИ

Для роботи з структурованими табличними даними, які розміщені в реляційній БД [SQLite](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiUyMOv1orvAhWDK3cKHac9DyMQFjAAegQIBxAD&url=https%3A%2F%2Fru.wikipedia.org%2Fwiki%2FSQLite&usg=AOvVaw1s-RMK2VC2tXyDZxZXDYDo) в Python передбачена стандартна бібліотека [sqlite3](https://docs.python.org/3/library/sqlite3.html)

In [1]:
# підключення бібліотеки
import sqlite3

#### ПРИКЛАД ВИЛУЧЕННЯ ДАНИХ З БД
Створити БД SQLite з ім'ям `FIT-3m`, та імпорувати в неї таблицю, що містить журнал групи `jornal`

In [2]:
# створюємо з'єднання з БД
conn = sqlite3.connect("FIT-3m.db")

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

In [3]:
# створюємо об'єкт `cursor`, що відповідає за реалізацію операцій з таблицями БД
cur = conn.cursor()

In [4]:
print(type(cur), end='\n\n')
print(dir(cur))

<class 'sqlite3.Cursor'>

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'close', 'connection', 'description', 'execute', 'executemany', 'executescript', 'fetchall', 'fetchmany', 'fetchone', 'lastrowid', 'row_factory', 'rowcount', 'setinputsizes', 'setoutputsize']


##### виведемо ВСІ записи таблиці `jornal`

In [5]:
# створимо інстанс для операції `SELECT`
students_cur = cur.execute("SELECT * FROM jornal")

# будуємо список записів за допомогою метода `fetchall()`
students_list = students_cur.fetchall()

_ = [print(x) for x in students_list]

OperationalError: no such table: jornal

#### ПРИКЛАД ЗБАГАЧЕННЯ ДАНИХ З ЗОВНІШНЬОГО ДЖЕРЕЛА

Додати до списка студентів їх оцінки за вступне тестування, які розташовані в csv файлі `test_result.csv`

імпортуємо `test_result.csv` в робочу БД як таблицю `test`

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

##### Cтворимо нову таблицю, яка буде містити вміст таблиці `jornal` та поле оцінок з таблиці `test`



In [6]:
# створимо вираз на побудову пустої таблиці `jornal_test` з відповідними полями
sql_stmt = "CREATE TABLE IF NOT EXISTS jornal_test \
              (Surname TEXT,         \
               Name TEXT,            \
               Email TEXT,           \
               Test_result INT);"

# виконаемо запит на створення таблиці
cur.execute(sql_stmt)

# перевіримо стан виконання операції - виведемо список таблиць
tables_list = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables_list.fetchall()


[('jornal_test',)]

In [7]:
# створимо запит на об'єднання таблиць 'jornal' та 'test' по прізвищу студента
sql_stmt = "SELECT jornal.*, test.\"Оцінка/100\" FROM  \
            jornal LEFT JOIN test ON jornal.Surname = test.\"Ім\'я\" "

# для перевірки виконаємо і виведемо результати запиту
cur.execute(sql_stmt).fetchall()

OperationalError: no such table: jornal

In [ ]:
# заповнимо пусту таблицю результатми  запиту на злиття
_ = cur.execute("INSERT INTO jornal_test " + sql_stmt)

In [ ]:
# збережемо вміст нової таблиці в список
jornal_test_list = cur.execute("SELECT * FROM jornal_test").fetchall()

_ = [print(x) for x in jornal_test_list]

In [ ]:
# завершимо всі транзакції
conn.commit()

# закріємо з'єднання
conn.close()

#### ПРИКЛАД  ДЕСКРИПТИВНОГО АНАЛІЗУ ДАНИХ

Провести [попередній аналіз](https://ru.wikipedia.org/wiki/Описательная_статистика) отриманих даних з ціллю виявленя відхилень, помилок та інших непридатних даних.

Сведемо результати аналізу в таблицю, яка має наступний вигляд:

№| Показчик | Значення
:--:|:-------|-------:
1| кількість спостережень | xx 
2| кількість пустих значень | xx
4| середній бал|  xx.x
5| максимальний бал | xx
6| мінімальний бал  | xx
7| стандартне відхилення | xx.x
8| розмах вариації  | xx


In [ ]:
# підключимо бібліотеку 'numpy' і дамо їй аліас 'np'
import numpy as np

ознайомитись з призначенням, можливостями та основним функціями [бібліотеки numpy](https://numpy.org) 

In [ ]:
# перетворимо список студетів в numpay матрицю
jornal_test_arr = np.array(jornal_test_list)

In [ ]:
print(type (jornal_test_arr), jornal_test_arr.shape)

In [ ]:
# створимо масив с результатами тестів 
test_result = np.delete(jornal_test_arr, [0,1,2], 1)

In [ ]:
# Значення 'None' - не є числом, то заміним його на 0
test_result[test_result == None] = 0

In [ ]:
# будуємо словник з вихідними розрахунками
result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)]) 
    }

In [ ]:
result_dict

In [ ]:
# вивести шапку
print(
"""
=======================================
№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     
=======================================
"""      
      )

# вивести результати аналізу датасета
i = 1
for key, value in result_dict.items():
    print (f'{i:<3} {key:<25}  {value}')
    i += 1

### ІНДИВІДУАЛЬНЕ ЗАВДАННЯ

In [ ]:
Користуючись результатами, що отримані в [Лабораторній роботі № 3](https://shkliarskiy.moodlecloud.com/mod/page/view.php?id=1193) виконати процедури видобутку, збагачення та попереднього аналізу даних.

__Постановка__: В 3-й лабораторній роботі отримано показчик, що характеризує окрему властивість квартири (ціна, метраж та ін.). 
Необхідно:
1. Відобразити цей показчик на адресу квартири, яка знаходиться в файлі `street_name.csv`, тобто побудувати новий файл `street_flat_propery.csv` якій містить 2 колонки: _<назва вулиці> <ваш показчик>_
2. На основі цьго файлу зробити відповідну таблицю в _SQLite_ та збагатити цю дані в цій таблиці назвою району де розташована квартира (дані по районах додаються) 
3. Провести попередній аналіз отриманих даних - вивести первинні статистики показчика по районах (_середне_ , _мінімальне_ , _максимальне_ і т.д. - див. приклад вище)

In [47]:
# Злиття файлу з показчиком та файлу з назвами вулиць в новий файл

YOUR_FILE = 'aprt_total_price.txt'
with open(YOUR_FILE) as f1, \
     open('street_name.csv') as f2, \
     open('street_flat_property.csv', 'w') as f3:
    
    for x, y in zip(f1, f2):
        line = f'{x[:-1].strip()},{y[:-1].strip()}\n'
        print(line)
        f3.write(line)

price,street

115000.0,Омеляновича-Павленка

75000.0,

140000.0,Ованеса Туманяна

140000.0,Петропавлівська

160000.0,Дніпровська

870000.0,Саксаганського

14883863.0,Івана Мазепи

220000.0,Дніпровська

133000.0,Ділова

69000.0,Новодарницька

169999.0,Ревуцького

212000.0,Липківського

83500.0,Бендукідзе

75000.0,Введенська

65000.0,Виноградний

82000.0,Кайсарова

207000.0,Ахматової

51990.0,Радченка

57999.0,Дніпровська

57000.0,Ділова

95000.0,Бориспільська

50000.0,Болсуновська

95000.0,Українки

46860.0,Антонова

38500.0,Омеляновича-Павленка

110000.0,

269000.0,Ованеса Туманяна

1983000.0,Петропавлівська

119000.0,Дніпровська

210000.0,Саксаганського

310000.0,Івана Мазепи

93000.0,Дніпровська

98000.0,Ділова

78000.0,Новодарницька

68000.0,Ревуцького

77000.0,Липківського

176000.0,Бендукідзе

5572000.0,Введенська

129000.0,Виноградний

136000.0,Кайсарова

71000.0,Ахматової

108000.0,Радченка

175000.0,Дніпровська

45006.0,Ділова

78000.0,Бориспільська

65000.0,Болсуновська

33960

#### імпортувати в БД отриманий файл 'street_flat_propery.csv' через CLI
- __sqlite3 districts.db__ - запустити SQLite та відкрити БД
- __.mode csv__ - перевести БД для роботи з csv-файлами
- __.import street_flat_propery.csv street_flat_propery__ - створює таблицю _street_flat_propery_ та завантажує в неї вміст файлу _street_flat_propery.csv_
- __SELECT * FROM street_flat_propery LIMIT 10;__ - вивести 10 записів для перевірки
- __.quit__ - завершити роботу з CLI SQLite



In [56]:
# Підключитись до БД 'districts.db' в якій є прив'язка вулиць до районів 
conn = sqlite3.connect("districts.db")

# створити відповідний курсор
cur  = conn.cursor()

In [57]:
# для перевірки виведіть 10 записів з таблиці 'street_district'
rows = cur.execute("SELECT * FROM street_district LIMIT 10")

rows_list = rows.fetchall()

_ = [print(x) for x in rows_list]

('9-го Травня', 'вулиця', 'Святошинський')
('Абрикосова', 'вулиця', 'Оболонський')
('Абрикосова', 'вулиця', 'Дарницький')
('Абрикосова', 'вулиця', 'Подільський')
('Августина Волошина', 'вулиця', 'Солом’янський')
('Авдєєнка Генерала', 'вулиця', 'Святошинський')
('Автозаводська', 'вулиця', 'Оболонський')
('Автозаводський ', 'провулок', 'Оболонський')
('Автопаркова', 'вулиця', 'Дарницький')
('Авторемонтна', 'вулиця', "Солом'янський")


In [58]:
# побудувати запит на створення таблиці 'result' з 3-ма полями:
#     district - текстове
#     street - текстове
#     property - числове
stmt = "CREATE TABLE IF NOT EXISTS result \
              (district TEXT,         \
               street TEXT,            \
               property INT);"


In [59]:
# виконати запит
cur.execute(stmt)

In [60]:
# перевірити схему таблиці
cur.execute ("SELECT name FROM PRAGMA_TABLE_INFO('result');").fetchall()

[('district',), ('street',), ('property',)]

In [61]:
# створити запит на злиття таблиці 'street_flat_propery' та 'street_district' 
# по полю 'street'
stmt_join = "SELECT street_district.district, street_flat_property.* FROM \
             street_flat_property LEFT JOIN street_district ON street_flat_property.street = street_district.street"

In [63]:
# перевірити запит
cur.execute(stmt_join).fetchall()

[(None, 115000.0, 'Омеляновича-Павленка'),
 (None, 75000.0, None),
 (None, 140000.0, 'Ованеса Туманяна'),
 ('Оболонський', 140000.0, 'Петропавлівська'),
 ('Подільський', 140000.0, 'Петропавлівська'),
 ('Оболонський', 160000.0, 'Дніпровська'),
 ('Голосіївський, Печерський, Шевченківський', 870000.0, 'Саксаганського'),
 (None, 14883863.0, 'Івана Мазепи'),
 ('Оболонський', 220000.0, 'Дніпровська'),
 ('Голосіївський, Печерський', 133000.0, 'Ділова'),
 (None, 69000.0, 'Новодарницька'),
 ('Дарницький', 169999.0, 'Ревуцького'),
 (None, 212000.0, 'Липківського'),
 (None, 83500.0, 'Бендукідзе'),
 ('Подільський', 75000.0, 'Введенська'),
 ('Печерський', 65000.0, 'Виноградний'),
 ("Голосіївський,Солом'янський", 82000.0, 'Кайсарова'),
 (None, 207000.0, 'Ахматової'),
 (None, 51990.0, 'Радченка'),
 ('Оболонський', 57999.0, 'Дніпровська'),
 ('Голосіївський, Печерський', 57000.0, 'Ділова'),
 ('Дарницький', 95000.0, 'Бориспільська'),
 ('Печерський', 50000.0, 'Болсуновська'),
 (None, 95000.0, 'Українки')

In [70]:
# створити запит для наповнення таблиці `result` результатом запиту на об'єднання
stmt_insert = ("INSERT INTO result " + stmt_join)

In [71]:
# виконати запит на заповнення
cur.execute(stmt_insert)

In [72]:
# сберегти вміст створеної таблиці у вигляді списка
result_list = cur.execute("SELECT * FROM result").fetchall()

_ = [print(x) for x in result_list]

(None, '115000.0', 'Омеляновича-Павленка')
(None, '75000.0', None)
(None, '140000.0', 'Ованеса Туманяна')
('Оболонський', '140000.0', 'Петропавлівська')
('Подільський', '140000.0', 'Петропавлівська')
('Оболонський', '160000.0', 'Дніпровська')
('Голосіївський, Печерський, Шевченківський', '870000.0', 'Саксаганського')
(None, '14883863.0', 'Івана Мазепи')
('Оболонський', '220000.0', 'Дніпровська')
('Голосіївський, Печерський', '133000.0', 'Ділова')
(None, '69000.0', 'Новодарницька')
('Дарницький', '169999.0', 'Ревуцького')
(None, '212000.0', 'Липківського')
(None, '83500.0', 'Бендукідзе')
('Подільський', '75000.0', 'Введенська')
('Печерський', '65000.0', 'Виноградний')
("Голосіївський,Солом'янський", '82000.0', 'Кайсарова')
(None, '207000.0', 'Ахматової')
(None, '51990.0', 'Радченка')
('Оболонський', '57999.0', 'Дніпровська')
('Голосіївський, Печерський', '57000.0', 'Ділова')
('Дарницький', '95000.0', 'Бориспільська')
('Печерський', '50000.0', 'Болсуновська')
(None, '95000.0', 'Українки'

#### для аналізу скористатися бібліотекою NumPy

In [73]:
import numpy as np

In [86]:
# створити np-масив з списку `result_list`
result_arr = np.delete(result_list, [0,2], 1)

result_arr[result_arr == None] = 0
result_arr = [float(el[0]) for el in result_arr]

In [90]:
# побудувати словник для результатів аналізу
result_dict = {
    "кількість спостережень"   : len(result_arr)  ,
    "середнє значення"         : round(np.mean(result_arr),1) ,
    "максимальне значення"     : np.max(result_arr) ,
    "мінімальне значення"      : np.min(result_arr) ,
    "стандартне відхилення"    : round(np.std(result_arr),1) ,
    "розмах вариації"          : np.max(result_arr) - np.min(result_arr)
    }
#  [np.nonzero(result_arr)]
print(result_dict)

{'кількість спостережень': 1622, 'середнє значення': 356496.8, 'максимальне значення': 14883863.0, 'мінімальне значення': 27500.0, 'стандартне відхилення': 1009040.0, 'розмах вариації': 14856363.0}


In [91]:
# вивести шапку
print(
"""
================================================
№  :   РАЙОН        : ПОКАЗЧИК        : ЗНАЧЕННЯ     
================================================
"""      
      )

# вивести результати аналізу датасета
i = 1
for key, value in result_dict.items():
    print (f'{i:<3} {key:<25}  {value}')
    i += 1


№  :   РАЙОН        : ПОКАЗЧИК        : ЗНАЧЕННЯ     

1   кількість спостережень     1622
2   середнє значення           356496.8
3   максимальне значення       14883863.0
4   мінімальне значення        27500.0
5   стандартне відхилення      1009040.0
6   розмах вариації            14856363.0
